In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [3]:
import numpy as np
from Scratch import metadata_loader

md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
md = md[md["distance"] == max_distance]
md = md.sort_values(by='backend_name', ascending=False)

md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
76,2023-10-29 14:47:58.814875+01:00,bigger_rep_codes,ibm_sherbrooke,cmz653m3r3vg008wf9j0,[],1111.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,30.0,30,1,_is_hex=True,Run bigger Repetition codes
75,2023-10-29 14:47:43.903639+01:00,bigger_rep_codes,ibm_sherbrooke,cmz64zvvcq70008qdxcg,[],1111.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,30.0,30,0,_is_hex=True,Run bigger Repetition codes


In [4]:
memories = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()

memories

KeyboardInterrupt: 

In [ ]:
import pickle
from soft_info import get_repcode_layout, get_KDEs, get_counts
from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit_qec.decoders import UnionFindDecoder

layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend("ibm_sherbrooke"), _is_hex=True)

kde_dict, scaler_dict = get_KDEs(provider, 'ibm_sherbrooke', layout, bandwidths=0.2, plot=False)
print(kde_dict)

#code = RepetitionCodeCircuit(max_distance, max_distance)
#decoder = UnionFindDecoder(code)

with open(f'dist_{max_distance}_decoder.pkl', 'rb') as f:
    decoder = pickle.load(f)

{114: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 116: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 118: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 120: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 122: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 104: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 102: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 100: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 98: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 96: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 94: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 75: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 77: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 79: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 81: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 8

In [17]:
from soft_info import get_counts

counts = get_counts(memories['mmr_log_0'], kde_dict, scaler_dict, layout, max_distance, verbose=True)

Processing 1111 shots: 100%|██████████| 1111/1111 [11:38<00:00,  1.59it/s]


In [14]:
from soft_info import soft_reweight, get_counts, rx_draw_2D
from tqdm import tqdm

VERBOSE = False

memory = memories['mmr_log_0']

logical_counts = {'0':0, '1':0}
for shot in tqdm(range(len(memory))[:100]):
    
    with open(f"dist_{max_distance}_decoder.pkl", 'rb') as f: # Reload the decoder each time to reset the graph
        decoder = pickle.load(f)

    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    if VERBOSE:
        print("Decoding graph before reweighting:")
        rx_draw_2D(decoder.graph)

    reweighted_graph = soft_reweight(decoder, IQ_data, kde_dict, scaler_dict, layout)
    decoder.graph = reweighted_graph

    if VERBOSE:
        print("Decoding graph after reweighting:")
        rx_draw_2D(decoder.graph)

    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    
    if VERBOSE:
        print("\nmeas_str:", count_key)
        print("flipped_qubit_dict", flipped_qubit_dict)
        print("logical:", logical, "\n")
        if logical[0] == 1:
            print("------------ Wrong logical! ------------\n")

    logical_counts[f"{logical[0]}"] += 1

print( "\nLogical_counts:", logical_counts)


100%|██████████| 20/20 [00:38<00:00,  1.93s/it]


Logical_counts: {'0': 20, '1': 0}


In [15]:
from soft_info import soft_reweight, get_counts, rx_draw_2D
from tqdm import tqdm

VERBOSE = False

memory = memories['mmr_log_0']

logical_counts = {'0':0, '1':0}
for shot in tqdm(range(len(memory))[:20]):
    
    with open(f"dist_{max_distance}_decoder.pkl", 'rb') as f: # Reload the decoder each time to reset the graph
        decoder = pickle.load(f)

    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    if VERBOSE:
        print("Decoding graph before reweighting:")
        rx_draw_2D(decoder.graph)

    #reweighted_graph = soft_reweight(decoder, IQ_data, kde_dict, scaler_dict, layout)
    #decoder.graph = reweighted_graph

    if VERBOSE:
        print("Decoding graph after reweighting:")
        rx_draw_2D(decoder.graph)

    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    
    if VERBOSE:
        print("\nmeas_str:", count_key)
        print("flipped_qubit_dict", flipped_qubit_dict)
        print("logical:", logical, "\n")
        if logical[0] == 1:
            print("------------ Wrong logical! ------------\n")

    logical_counts[f"{logical[0]}"] += 1

print( "\nLogical_counts:", logical_counts)


100%|██████████| 20/20 [00:15<00:00,  1.29it/s]


Logical_counts: {'0': 20, '1': 0}
